In [ ]:
import pandas as pd
df = pd.read_csv(filepath_or_buffer='/kaggle/input/commonlit-texts/commonlit_texts.csv')
df['grade_int'] = df['grade'].apply(func=lambda x: int(x.replace('th Grade', '').replace('rd Grade', '')))
df.head()

In [ ]:
df.info()

In [ ]:
df.nunique()

In [ ]:
from plotly.express import bar
grade_df = df[['grade', 'grade_int']].drop_duplicates(ignore_index=True).sort_values(by='grade_int')
bar(data_frame=df, x='grade', category_orders={'grade': grade_df['grade'].values.tolist()})